### **Import Libraries**

In [1]:
import requests
import pandas as pd
import numpy as np 
import datetime

In [10]:
url = "https://api.spacexdata.com/v4/launches/past"

response = requests.get(url)
get_response = response.json()

In [3]:
def BoosterVersion(data): 
    for x in data["rocket"]: 
        if x: 
            response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
            BoosterVersion.append(response["name"])

In [8]:
def LaunchSite(data): 
    for x in data["launchpad"]: 
        if x: 
            response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
            Longitude.append(response["longitude"])
            Latitude.append(response["latitude"])
            LaunchSite.append(response["name"])

In [9]:
def PayloadData(data): 
    for load in data["payloads"]: 
        if load: 
            response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
            PayloadMass.append(response["mass_kg"])
            Orbit.append(response["orbit"])
            

In [10]:
def CoreData(data): 
    for core in data["cores"]: 
        if core["core"] != None: 
            response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
            Block.append(response["block"])
            ReusedCount.append(response["reuse_count"])
            Serial.append(response["serial"])
        
        else: 
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
        
        Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
        Flights.append(core['flight'])
        GridFins.append(core['gridfins'])
        Reused.append(core['reused'])
        Legs.append(core['legs'])
        LandingPad.append(core['landpad'])

In [11]:
url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(url)

In [ ]:
print(response.content)